### Packages

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image

### 이미지 데이터 전처리

In [ ]:
# 데이터 경로 설정
fun_path = "/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/fun"          # 즐거운
healing_path = "/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/healing"  # 힐링
mood_path = "/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/mood"        # 감성적인

In [ ]:
# 이미지 크기 및 채널 설정
img_size = (224, 224)
channels = 3

In [ ]:
# 이미지 데이터 로드 및 전처리
def load_and_preprocess_data(image_paths, label, img_size=(224, 224)):
    data = []
    labels = []

    for img_path in image_paths:
        try:
            # Skip non-image files
            if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            img = Image.open(img_path)
            img = img.convert('RGB')  # Ensure the image is in RGB mode
            img = img.resize(img_size)
            img_array = np.array(img)  # Convert PIL Image to NumPy array
            img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

            data.append(img_array)
            labels.append(label)

        except Exception as e:
            print(f"Error processing image {img_path}: {str(e)}")

    return np.array(data), np.array(labels)

In [ ]:
fun_images = [os.path.join(fun_path, img) for img in os.listdir(fun_path)]
healing_images = [os.path.join(healing_path, img) for img in os.listdir(healing_path)]
mood_images = [os.path.join(mood_path, img) for img in os.listdir(mood_path)]

fun_data, fun_labels = load_and_preprocess_data(fun_images, label=0)
healing_data, healing_labels = load_and_preprocess_data(healing_images, label=1)
mood_data, mood_labels = load_and_preprocess_data(mood_images, label=2)

### 학습/테스트 데이터 분할

In [ ]:
# 데이터 합치기 및 레이블 변환
X = np.concatenate([fun_data, healing_data, mood_data], axis=0)
y = np.concatenate([fun_labels, healing_labels, mood_labels], axis=0)

# 레이블을 범주형 형태로 변환
y = to_categorical(y, num_classes=3)

In [ ]:
# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 모델링
- Test Loss: 2.2566311359405518, Test Accuracy: 0.5625

In [ ]:
# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 분류 클래스 수에 맞게 설정

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
76/76 [==============================] - 23s 293ms/step - loss: 1.6908 - accuracy: 0.3943 - val_loss: 1.0943 - val_accuracy: 0.4095
Epoch 2/10
76/76 [==============================] - 22s 288ms/step - loss: 0.9739 - accuracy: 0.5275 - val_loss: 1.0279 - val_accuracy: 0.4786
Epoch 3/10
76/76 [==============================] - 24s 316ms/step - loss: 0.7961 - accuracy: 0.6439 - val_loss: 0.9692 - val_accuracy: 0.5691
Epoch 4/10
76/76 [==============================] - 23s 300ms/step - loss: 0.5061 - accuracy: 0.8063 - val_loss: 1.0818 - val_accuracy: 0.5789
Epoch 5/10
76/76 [==============================] - 22s 291ms/step - loss: 0.2708 - accuracy: 0.9116 - val_loss: 1.4509 - val_accuracy: 0.5691
Epoch 6/10
76/76 [==============================] - 22s 293ms/step - loss: 0.1672 - accuracy: 0.9589 - val_loss: 1.5598 - val_accuracy: 0.5839
Epoch 7/10
76/76 [==============================] - 22s 284ms/step - loss: 0.1075 - accuracy: 0.9766 - val_loss: 2.3430 - val_accuracy: 0.5214

In [ ]:
# 테스트 데이터에 대한 예측 수행
predictions = model.predict(X_test)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

19/19 [==============================] - 1s 77ms/step - loss: 2.2566 - accuracy: 0.5625
Test Loss: 2.2566311359405518, Test Accuracy: 0.5625


### 예시 이미지로 테스트

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 로드 및 전처리
img_path = r"/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/trial/fun_trial_1.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # 이미지 정규화

# 모델에 입력 이미지 전달하여 예측 수행
predictions = model.predict(img_array)

# 예측 결과 확인
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class[0])

class_mapping = {0: "fun", 1: "healing", 2: "mood"}
predicted_label = class_mapping[predicted_class[0]]
print("Predicted label:", predicted_label)

1/1 [==============================] - 0s 55ms/step
Predicted class: 2
Predicted label: mood


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 로드 및 전처리
img_path = r"/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/trial/healing_trial_1.jpeg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # 이미지 정규화

# 모델에 입력 이미지 전달하여 예측 수행
predictions = model.predict(img_array)

# 예측 결과 확인
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class[0])

class_mapping = {0: "fun", 1: "healing", 2: "mood"}
predicted_label = class_mapping[predicted_class[0]]
print("Predicted label:", predicted_label)

1/1 [==============================] - 0s 23ms/step
Predicted class: 1
Predicted label: healing


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 로드 및 전처리
img_path = r"/Users/euijinlee/KDT_DATA/파이널프로젝트/data1/trial/mood_trial_1.jpeg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # 이미지 정규화

# 모델에 입력 이미지 전달하여 예측 수행
predictions = model.predict(img_array)

# 예측 결과 확인
predicted_class = np.argmax(predictions, axis=1)
print("Predicted class:", predicted_class[0])

class_mapping = {0: "fun", 1: "healing", 2: "mood"}
predicted_label = class_mapping[predicted_class[0]]
print("Predicted label:", predicted_label)

1/1 [==============================] - 0s 20ms/step
Predicted class: 0
Predicted label: fun


### Save

In [ ]:
#모델 저장
model.save('cnn_02_leeeug.h5')

C:\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
